In [ ]:
import torch

In [ ]:
from torch import nn

class ConvNet(nn.Module):
    
    def __init__(self,kernel_size:int,number_of_clases:int):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=kernel_size),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,kernel_size=kernel_size),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64,128,kernel_size=kernel_size),
            nn.ReLU(),
            nn.Flatten()
        )
        self.classifiyer = nn.Sequential(
            nn.Linear(128*kernel_size*kernel_size,110),
            nn.ReLU(),
            nn.Linear(110,number_of_clases),
            nn.Softmax(1)
        )

    def forward(self,x):
        x = self.encoder(x)
        x = self.classifiyer(x)

        return x

In [ ]:
img = torch.rand((20,1,28,28)).cuda()
# img =img[None,:].cuda()
lbl = torch.randint(0,25,(20,1)).cuda()
lbl.shape

In [ ]:
loss = torch.nn.CrossEntropyLoss()
conv_classfier = ConvNet(3,26).cuda()

In [ ]:
logits = conv_classfier(img)
logits.shape

In [ ]:
loss(logits,lbl)

In [ ]:
import kagglehub
from pandas import read_csv,concat
# Import the Dataset from kaggle and load on a Tensor
Nist_dataset = kagglehub.dataset_download("sachinpatel21/az-handwritten-alphabets-in-csv-format")

chunksize = 250

# the list that contains all the dataframes
list_of_dataframes = []

for df in read_csv(Nist_dataset+"/A_Z Handwritten Data.csv", chunksize=chunksize):
    # process your data frame here
    # then add the current data frame into the list
    list_of_dataframes.append(df)

# if you want all the dataframes together, here it is
handwritten_alphabet = concat(list_of_dataframes)



# handwritten_alphabet = read_csv(Nist_dataset+"/A_Z Handwritten Data.csv")
# print(handwritten_alphabet)
handwritten_alphabet = torch.Tensor(handwritten_alphabet.values).cuda()

In [ ]:
#Create a Dataset Class to load the Images and labels
class image_dataset(torch.utils.data.Dataset):
    def __init__(self,images:torch.Tensor,labels:torch.Tensor):
        if len(images[:]) != len(labels):
            raise IndexError("The size must be the same")
        self.images = images
        self.labels = labels

    def __getitem__(self, index):
        
        image = self.images[None,index]
        label = self.labels[index]

        if type(index) == list:
            image = image.swapaxes(0,1)
            # label = label.swapaxes(0,1)

        return image, label
    
    def __len__(self):
        return len(self.labels)

In [ ]:
# Split The data on Images and Labels
# handwritten_alphabet[:,0]
handwritten_alphabet_classes = handwritten_alphabet[:,0].long()
handwritten_alphabet_images = handwritten_alphabet[:,1:].reshape((-1,28,28))

In [ ]:
# Create and Split the dataset on Train, Validation and Test Datasets
image_dataset = image_dataset(handwritten_alphabet_images,handwritten_alphabet_classes)

train,val,test = torch.utils.data.random_split(image_dataset,[0.8,0.1,0.1])


In [ ]:
handwritten_alphabet_classes[0:2].shape

In [ ]:
train[0:55624][0].shape

In [ ]:
loader_prueba = torch.utils.data.DataLoader(
    dataset=train,
    shuffle=True,
    batch_size=20
)
_,lbls = next(iter(loader_prueba))
lbls.shape